# Imports

In [9]:
import pandas as pd
import pyreadstat as spss
import os

In [2]:
# DATA FOR 2014
DataFile='/Users/fipm/OneDrive/Research/Mineduc/BaseDatosPruebas/2014/2014 - 3o primaria/2014 - 3ro Primaria - Versión Final.sav'
df, meta = spss.read_sav(DataFile, encoding='LATIN1', apply_value_formats=True)


In [12]:
db = df.filter(['Cod_Estab','measure_L','measure_M'])
db.to_csv('/Users/fipm/OneDrive/GitHubProjects/grade_progression/raw_data/data.csv',
          index=False)

# starts main program

In [13]:
# Loading data
file_path = os.path.join('..','raw_data','data.csv')
db = pd.read_csv(file_path)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19856 entries, 0 to 19855
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   Cod_Estab  19856 non-null  object 
 1   measure_L  19837 non-null  float64
 2   measure_M  19830 non-null  float64
dtypes: float64(2), object(1)
memory usage: 465.5+ KB
